# NOTE:

Code is not optimized yet - has a lot of for loops and stuff. Was just trying to get it to work. Once we have it working end to end, I'll see where I can switch to numpy.

## Load pdfs as text
Iterate over the pdfs and convert each file to a string. Store the strings as a list.

## Set Up Code
Here I just install and import some libraries, set up IAM roles, specify the S3 bucket, etc.

In [1]:
%%capture
!pip install torch==1.8.1+cu102 torchvision==0.9.1+cu102 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html # need to re-install everytime I shut off the instance
!pip install datasets
!pip install huggingface_hub
!pip install -U transformers
!pip install accelerate
!pip install PyPDF2

In [2]:
from sagemaker import get_execution_role
role = get_execution_role() # set IAM role

In [3]:
import PyPDF2, re
from io import BytesIO
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
torch.cuda.is_available()

True

In [5]:
tr=pd.read_json('train.json', lines=True)
te=pd.read_json('test.json', lines=True)
complete=pd.read_json('completeDataset.json', lines=True)

In [6]:
test_dataset = Dataset.from_pandas(te)
print(test_dataset)
train_dataset = Dataset.from_pandas(tr)
print(train_dataset)
complete_dataset = Dataset.from_pandas(complete)
print(complete_dataset)

Dataset({
    features: ['Title', 'Text', 'Summary'],
    num_rows: 34
})
Dataset({
    features: ['Title', 'Text', 'Summary'],
    num_rows: 136
})
Dataset({
    features: ['Title', 'Text', 'Summary'],
    num_rows: 170
})


In [7]:
tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-led-base-16384")
max_input_length = 8192
max_output_length = 512
batch_size = 2
padding = "max_length" 

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Define a function that pre-processes data.

In [8]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["Text"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["Summary"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [9]:
train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["Title", "Text", "Summary"]
)

test_dataset = test_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["Title", "Text", "Summary"],
)

train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

test_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/17 [00:00<?, ?ba/s]

In [10]:
led = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-led-base-16384", gradient_checkpointing=True, use_cache=False)

# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

Downloading:   0%|          | 0.00/648M [00:00<?, ?B/s]

In [11]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 KB 4.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=ef96ac0c4c0eb8792274048385dc631ec7768c35c9bd5f452788649fa43ff20d
  Stored in directory: /home/ec2-user/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge_score
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [12]:
rouge = load_metric("rouge")

/tmp/ipykernel_3903/4132584981.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


In [13]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [14]:
# enable fp16 apex training
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # fp16=True,
    output_dir="./",
    logging_steps=5,
    eval_steps=10,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
)

In [15]:
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [16]:
# del cleanedTexts,trainSet, testSet, PyPDF2, i, j, contentsPDF, BytesIO, completeSet, complete_dataset, englishOnly, filepaths, finalTextString, fs, n, pattern, re, result, summaries

In [17]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [18]:
trainer.train()

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 136
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 17
  Number of trainable parameters = 161844480
You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.206600,3.206204,0.118800,0.135800,0.117300


***** Running Evaluation *****
  Num examples = 34
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10/config.json
Model weights saved in ./checkpoint-10/pytorch_model.bin
tokenizer config file saved in ./checkpoint-10/tokenizer_config.json
Special tokens file saved in ./checkpoint-10/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=17, training_loss=3.316396573010613, metrics={'train_runtime': 2962.2887, 'train_samples_per_second': 0.046, 'train_steps_per_second': 0.006, 'total_flos': 734455715069952.0, 'train_loss': 3.316396573010613, 'epoch': 1.0})

In [19]:
trainer.save_model('led_summarizer/model_data')

Saving model checkpoint to led_summarizer/model_data
Configuration saved in led_summarizer/model_data/config.json
Model weights saved in led_summarizer/model_data/pytorch_model.bin
tokenizer config file saved in led_summarizer/model_data/tokenizer_config.json
Special tokens file saved in led_summarizer/model_data/special_tokens_map.json


Upload Model Artifacts to S3

In [20]:
# # Package model into a TAR file 
# import tarfile
   
# tarfile_name='model_data.tar.gz'
   
# with tarfile.open(tarfile_name, mode='w:gz') as archive:
#     archive.add('led_summarizer/model_data')

In [21]:
# Package model into a TAR file 
import tarfile
   
tarfile_name='model_data.tar.gz'
   
with tarfile.open(tarfile_name, mode='w:gz') as archive:
    archive.add('checkpoint-10')

In [22]:
!aws s3 cp model_data.tar.gz s3://sagemaker-legal-summarizer-model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
upload: ./model_data.tar.gz to s3://sagemaker-legal-summarizer-model/model_data.tar.gz


In [27]:
# from sagemaker.pytorch import PyTorchModel
from sagemaker.huggingface.model import HuggingFaceModel

huggingface_model = HuggingFaceModel(
   model_data="s3://sagemaker-legal-summarizer-model/model_data.tar.gz",  # path to your trained SageMaker model
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.12.3",                           # Transformers version used
   pytorch_version="1.9.1",                                # PyTorch version used
   env={'MMS_DEFAULT_RESPONSE_TIMEOUT': '500', "HF_TASK":"summarization", "HF_MODEL_ID":"nsi319/legal-led-base-16384"
},
   py_version='py38',                                    # Python version used
)

In [28]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g4dn.2xlarge"
)

-----------!

In [29]:
input= {"inputs": "1 consultation paper on green and blue bonds as a mode of sustainable finance 1. Background 1.1 sustainable finance is the process of taking due account of environmental, social and governance (esg) considerations while making investment decisions in the financial sector, leading to increased longer-term investments into sustainable economic activities and projects.1 1.2 the need for sustainable finance was felt in the aftermath of repeated environmental disasters and extreme weather events, corporate finance being driven purely by profit motives and corporate governance norms being followed as the exception rather than the norm. This has been bolstered by the demand for sustainable avenues of investments from a section of the new- age investors as well as with the substantial rise in the prominence of the stakeholders theory over shareholders theory. 1.3 the cop 26 mandate: in the 2021 united nations climate change conference, more commonly referred to as cop26, held in glasgow, scotland, 197 countries, including india, have made enhanced commitments towards mitigating climate change and promising more climate finance for developing countries to adapt to climate impacts. This means that sustainable finance will be the mainstay of world business, which in turn, will mean more demand for funds in this domain. 2. Sebis regulatory regime on green bonds: 2.1 the concept of green debt securities was introduced under the erstwhile securities and exchange board of india (issue and listing of debt securities) regulations, 2008 vide circular dated may 30, 20172. 2.2 regulation 2(1)(q) of the securities and exchange board of india (issue and listing of non- convertible securities) regulations (ncs regulations) defines green debt security, which reads as under: green debt security means a debt security issued for raising funds that are to be utilised for project(s) and/or asset(s) falling under any of the following categories, subject to the conditions as may be specified by the board from time to time: (i) renewable and sustainable energy including wind, solar, bioenergy, other sources of energy which use clean technology, (ii) clean transportation including mass/public transportation, (iii) sustainable water management including clean and/or drinking water, water recycling, 1 https: //www. Worldbank. Org/en/topic/financialsector/brief/sustainable-finance 2 subsumed in the operational circular on the ncs regulations, 2021 please see chapter ix page 1 of 15 2 (iv) climate change adaptation, (v) energy efficiency including efficient and green buildings, (vi) sustainable waste management including recycling, waste to energy, efficient disposal of wastage, (vii) sustainable land use including sustainable forestry and agriculture, afforestation, (viii) biodiversity conservation, or (ix) a category as may be specified by the board, from time to time 2.3 further, chapter ix of the operational circular on the ncs regulations, 2021 dated august 10, 2021, as amended on date, specifies the following with reference to issuers of green debt securities: a. Additional disclosure requirements in the offer document; b. Continuous disclosure requirements in annual report and financial results; c. Responsibilities of the issuer. 3. Mandate of this consultation paper 3.1. Since the framework of green debt securities was laid down by sebi, there have been multiple events in the sustainable finance space around the world, thereby necessitating a review in the indian context. 3.2. In this context, sebi, through this consultation paper, is seeking public comments on a proposed regulatory framework: a. To amplify the definition of green debt securities, b. To introduce the concept of blue bonds c. To reduce the compliance cost for issuers of green debt securities with while not creating any perverse incentives that may lead to greenwashing. 3.3. Suggestions are also solicited towards increasing avenues for sustainable finance in india, while considering indias unique goals of pursuing high growth with sustainable development. 3.4. This consultation paper follows a series of discussions held with multiple stakeholders, including the sub-committee formed under the corporate bonds securitization advisory committee (cobosac) of sebi. 4. Global scenario 4.1 issuance of green, social, sustainable and sustainability-linked (gsss) bonds as a percentage of total global bond issuance rose from roughly 2 per cent at the start of 2018 to a peak of over 12 per cent at the end of 20213. Prices continued to rise as the demand from investors exploded. Figures from data provider epfr, as given in the graph below, show that flows into esg and socially responsible investing bond funds rose from a net of $4.2bn in 2018 to $102bn in 2021. 3 https: //www. Ft. Com/content/32dbf37c-8ff5-436b-88f3-9873fc864a7b page 2 of 15 3 4.2 with economic growth slowing globally owing to high inflation, high commodity prices and higher interest rates issuers of bonds, both green and conventional, have pulled back. As the sustainable bonds are subject to the same headwinds as that of the broader market, green bond issuances in this year are expected to be roughly in line with 2021. That is a notable slowdown, given how quickly the market has been growing. Despite the above fact, rating agencies forecast that global issuance of gsss bonds will be around $1tn this year4. 5. Present fund raising through green bonds india 5.1. As per the economic survey 2019-205, climate bonds worth usd 10.3 billion were issued in the first half of 2019 in india. Climate bonds in india are focused on green bonds, which are specifically linked to climate-change mitigation, adaptation and resilience. 5.2. A number of government agencies have contributed to issuance of such bonds, particularly indian renewable energy development agency (ireda) and the indian railway finance corporation (irfc). 5.3. Currently, green bonds raised by various issuers are listed on india inx. Similarly, various issuers have also issued and listed their bonds on other international stock exchanges such as luxembourg se, singapore exchange, etc. 5.4. Indian companies raised nearly usd 7 billion through esg and green bonds in 2021, compared to usd 1.4 billion and usd 4 billion in 2020 and 2019, respectively. 5.5. As of february 12, 2020, the outstanding amount of green bonds in india was us$16.3 billion. 4 https: //www. Climatebonds. Net/2022/01/500bn-green-issuance-2021-social-and-sustainable-acceleration-annual-green-1tn- sight-market 5 https: //www. Indiabudget. Gov. In/budget2020-21/economicsurvey/index. Php page 3 of 15 4 5.6. As per economic survey 2020-21 and emerging markets green bonds report 2020, india occupies the 2nd spot (after china) in the cumulative emerging market green bond issuance, 2012-20206. Cumulative emerging market green bond issuance, 2012-2020 (us$ million) 5.7. The above graph shows that in most countries, green bond issuances by corporate entities vastly exceed the issuances by government/ sovereign agencies. The only exceptions are chile, poland, indonesia, hungary, egypt and argentina. 5.8. As on june 30, 2022, under the sebi framework for green bonds, 14 issuers have raised rs. 4,539 crore (about $ 0.5 billion). 6 https: //www. Indiabudget. Gov. In/budget2021-22/economicsurvey/index. Php page 4 of 15 5 5.9. As can be seen from above, there is a stark divergence between the amount raised by listing green bonds in indian jurisdictions versus foreign jurisdictions or autonomous jurisdictions like inx (india international exchange). It may be noted that issuers are finding it more attractive to list on bourses falling outside sebi framework. 5.10. Most of the green bonds issued by indian issuers are listed on offshore exchanges. Informal consultation with the industry participants attribute this to: - lack of demand for such bonds among the indian investors - favourable pricing in overseas market. 6. Scope for sustainable finance in india 6.1 at the cop-26 summit in glasgow, india had made the following commitments towards its climate change goals7: a. Raising non-fossil fuel based energy capacity to 500 gw. B. Lowering total projected carbon emission by one billion tons. C. Meeting 50% of the countrys energy needs through renewable sources. D. Reduce the carbon intensity of the economy to sub 45% level. E. Commitment to achieve net-zero emissions by 2070. 6.2 government of india has initiated the following schemes/ policies in pursuit of the goals towards cop-26 targets: a. Commission on air quality management (caqm) - phasing out of coal use in national capital region from january 2023, but exempting thermal power plants using low- sulphur coal. B. National biomass co-firing policy to use biomass like paddy stubble and cow-dung in coal power plants8. C. Mandate the use of flue gas desulphurization (fgd) techniques in both existing and upcoming thermal power plants. D. Government has targeted to achieve 100 million tonnes of coal gasification by 20309. E. National mission on transformative mobility and battery storage10. F. Productivity linked incentives (pli) scheme on advanced cell chemistry and battery storage11. G. India cooling action plan12. 7 https: //www. Mea. Gov. In/speeches- statements. Htm? Dtl/34466/national_statement_by_prime_minister_shri_narendra_modi_at_cop26_summit_in_glasgow 8 https: //powermin. Gov. In/sites/default/files/revised_biomass_policy_dtd_08102021. Pdf 9 https: //pib. Gov. In/pressreleaseshare. Aspx? Prid=1650096 10 https: //www. Niti. Gov. In/e-mobility-national-mission-transformative-mobility-and-battery-storage 11 https: //www. Niti. Gov. In/sites/default/files/2022-02/need-for-acc-energy-storage-in-india. Pdf 12 https: //www. Iea. Org/policies/7455-india-cooling-action-plan-icap page 5 of 15 6 6.3 views/ comments sought on: a. Whether the above mentioned initiatives offer any scope for financing through green bonds? B. Please offer comments on whether there are any additional policies/ schemes in line with the above objective, if any? 7. Potential in emerging areas 7.1. The sub-committee constituted by the cobosac had presented its report, which has covered areas like priority sector lending to green sectors, investment by institutional investors, tax recommendation for green bonds and updation of extant guidelines in line with updated green bond principles published by the international capital markets association13. 7.2. There is a need to further channel the investment in emerging areas like data centers which are receiving policy traction (draft data center policy). This field assumes importance in light of government push for data localization and the storage necessities for large volumes of data collected in india. Data centers need tremendous cooling facilities, some of which can be attractive avenues for sustainable finance. 7.3. New concepts like circular economy and regional resource based manufacturing are gaining currency, as a mode to balance job creation with inter-generational environmental equity. Though definitions of these need deeper analysis, there is a need for encouraging flow of green finance in these initiatives. The government of india has presented schemes like one district one product, zero effect zero defect and promotion of these goods through local post offices and railway stations. 7.4. Ecosystem services evaluation- the natural capital accounting and valuation of ecosystem services (ncaves) india forum is being organized by ministry of statistics and programme implementation (mospi) in collaboration with the united nations statistics division (unsd), european union and un environment. India has also implemented the system of environmental economic accounting (seea) which provides policy makers and decision-makers with information that accounts for the value nature contributes to our economies.14 7.5. Agritech companies investing in net-zero for agriculture: - agriculture and allied sectors like animal husbandry are responsible for over 85% of anthropogenic greenhouse gas emissions. Sustainable finance and technology in this sector can take out a major source of climate change. 7.6. National hydrogen mission- india is targeting to produce three-fourths of its hydrogen from renewable resources by 2050.15 7.7. Blue economy and ocean conservation.16 13 https: //www. Icmagroup. Org/sustainable-finance/the-principles-guidelines-and-handbooks/green-bond-principles-gbp/ 14 https: //seea. Un. Org/content/natural-capital-accounting-and-valuation-ecosystem-services-india 15 https: //pib. Gov. In/pressreleasepage. Aspx? Prid=1799067 16 https: //www. Worldbank. Org/en/news/infographic/2017/06/06/blue-economy page 6 of 15 7 8. Potential for blue bonds 8.1. According to the world bank, the blue economy is the sustainable use of ocean resources for economic growth, improved livelihoods, and jobs while preserving the health of ocean ecosystem. 17 european commission defines it as all economic activities related to oceans, seas and coasts. 8.2. For a country like india that has a 7500 km long coastline and 14,500 km of navigable inland waterways, the development of blue economy can serve as a growth catalyst in realizing the vision to become a $10 trillion economy by 2032. The blue economy currently comprises 4.1% of indias economy. 8.3. The government of india has kickstarted the initiative to develop the blue economy by announcing project sagarmala, a first of its kind port-led development programme in india. The sagarmala programme plans to bring in development around india's coastal regions based on the following four pillars: port modernization & new port development, port connectivity enhancement & port-linked industrialization. 8.4. The ministry of earth sciences (moes) has rolled out the draft blue economy policy, inviting suggestions and inputs from various stakeholders. 8.5. It emphasizes policies across several key sectors to achieve holistic growth of indias economy. It recognizes the following thematic areas: a. National accounting framework for the blue economy and ocean governance. B. Coastal marine spatial planning and tourism. C. Marine fisheries, aquaculture, and fish processing. D. Manufacturing, emerging industries, trade, technology, services, and skill development. E. Logistics, infrastructure and shipping, including trans-shipments. F. Coastal and deep-sea mining and offshore energy. G. Security, strategic dimensions, and international engagement. H. Indias exclusive economic zone of over 2 million square kilometres has a huge living and non-living resources with significant recoverable resources such as crude oil and natural gas. 9. Scope of blue bonds in india india has tremendous scope for deployment of blue bonds in various aspects of the blue economy, like: 9.1 oceanic resource mining: polymetallic nodules and methane hydrates are abundant in the indian ocean region. Mining of polymetallic nodules present in the seabed in the central indian ocean basin can help india improve availability of nickel, copper, cobalt and manganese. Through an agreement with the international seabed authority, india has a right to explore and mine polymetallic nodules over 750,000 square km.18 17 https: //www. Worldbank. Org/en/news/feature/2018/10/29/sovereign-blue-bond-issuance-frequently-asked- questions 18 https: //pib. Gov. In/newsite/printrelease. Aspx? Relid=170138 page 7 of 15 8 9.2 sustainable fishing: as the developing countries face new challenges at wto to phase out illegal, unreported and unregulated (iuu) fishing, it is time the country phases out exploitative fishing devices like bottom trawlers. India has an exclusive economic zone of 2.37 million sq. Km. 9.3 national offshore wind energy policy: ministry has set a target of 5.0 gw of offshore wind installations by 2022 and 30 gw by 2030 which has been issued to give confidence to the project developers in indian market. 9.4 coral degradation: india can invest in technologies like biorock to rejuvenate the corals along its coastline, by forming an artificial substrate. 9.5 india has acceded to the marine pollution (marpol) convention covering prevention of pollution of the marine environment by ships from operational or accidental causes. 9.6 geoengineering techniques like ocean fertilization or ocean nourishment is a type of climate engineering based on the purposeful introduction of plant nutrients to the upper ocean to increase marine food production and to remove carbon dioxide from the atmosphere. 9.7 blue flag beach is an eco-tourism model endeavouring to provide the tourists/ beachgoers clean and hygienic bathing water, facilities, a safe and healthy environment and sustainable development of the area.19 9.8 views/ comments sought on: a. Whether the above mentioned initiatives offer any scope for financing through blue bonds? B. Whether introducing coloured bonds (blue bonds for blue economy, yellow bonds for solar power) will help increase channels for funding to green projects? C. Please offer comments on whether any additional policies/ schemes, if any? 10. Recommendations made by the sub-committee of cobosac on green bonds the sub-committee has proposed the following amendments to sebi guidelines in order to align with the updated green bond principles (gbp) published by the international capital market association (icma): sr. Existing need for change proposed amendment no. Clause a. Definition of the extant sebi framework defines the following may be included green debt gds as debt security issued for in addition to the existing list security (gds) raising funds that are to be utilised for of eligible categories for regulation projects or assets falling under issuance of gds: 2(1)(q) of sebi certain categories. A. Pollution prevention and (issue and gbp 2021 have included two control (including reduction of listing of non- additional categories, namely, air emissions, greenhouse convertible pollution prevention and control and gas control, soil remediation, securities), waste 19 http: //blueflagindia. Org/ page 8 of 15 9 regulations, circular economy adapted products, prevention/minimization, 2021) as eligible green projects. Waste reduction, waste in order to align with gbp 2021 recycling and energy/ principles, these additional emission efficient waste to categories may be included. Energy). B. Circular economy adapted products, production technologies and processes (such as the design and introduction of reusable, recyclable and refurbished materials, components and products, circular tools and services) and/or certified eco efficient products. B. Utilisation of the extant sebi framework for gds in addition to the existing issue do not mandate disclosure of disclosure requirements, we proceeds temporary placement for the balance may include that, the issuer clause 1 of of unallocated net proceeds, if any. Shall disclose the intended chapter ix of gbp 2021 have recommended that types of temporary placement sebi circular the issuer should make known to for the balance of unallocated dated investors the intended types of net proceeds. December 17, temporary placement for the balance 2021 of unallocated net proceeds. In view of the same, the temporary utilization of proceeds raised for green debt securities may be specified in order to have better disclosure and increased transparency. C. Utilisation of the extant sebi framework for gds in addition to the existing issue do not mandate disclosure of disclosure requirements, we proceeds utilization of issue proceeds for each may include that, the clause 2 of debt security issued by an issuer. Utilisation of proceeds from chapter ix of gbp 2021 recommends that the each issue of gds made by sebi circular proceeds of green bonds can be an issuer shall be tracked and dated managed per bond (bond-by-bond disclosed separately (i. E. On a december 17, approach) or on an aggregated basis bond by bond approach) or on 2021 for multiple green bonds (portfolio an aggregated basis for approach). Multiple green bonds in order to have clear distinction (portfolio approach). Between the uses of proceeds of multiple issuances of green debt security by a single issuer for various page 9 of 15 10 green projects, bond by bond approach may be adopted. D. Disclosure the extant sebi framework for gds in addition to the existing requirements does not mandate disclosure of disclosure requirements for clause 2 of taxonomy, standards, certifications, selection of projects to be chapter ix of etc. Referred for project selection by financed through gds, we the sebi the issuer. May include that, the issuer circular dated gbp 2021 recommends the issuer to shall disclose any december 17, provide information, if relevant, on the taxonomies, green standards 2021 alignment of projects with official or or certifications, if referenced market-based taxonomies, related in the project selection. Eligibility criteria, including if issuers should provide applicable, exclusion criteria; and information regarding the also disclose any green standards or alignment of projects with said certifications referenced in project taxonomies, related eligibility selection. Criteria, including if applicable, in line with recommendations of gbp exclusion criteria. 2021, the aforementioned disclosure may be made applicable to issuers of gds. E. Disclosure with respect to refinancing of assets in order to enhance disclosure requirements from proceeds of gds, the extant requirements for refinancing clause 1 of sebi framework requires the issuer of projects through gds, chapter ix of to disclose details of projects and/ or clause 1.4 of chapter ix of the sebi assets or areas where the issuer the sebi circular dated circular dated proposed to utilize the proceeds of december 17, 2021 may be december 17, the issue of gds including towards amended as follows: 2021 refinancing of existing green projects a. Details of identified and/or assets, if any. Project(s) and/or asset(s) gbp 2021 recommends the where the issuer proposes to following: utilise/ allocate the proceeds - in the event that all or a proportion of the issue of gds. Of the proceeds are or may be used b. In case a portion of the for refinancing, issuers provide an proceeds of the issue are estimate of the share of financing vs. Proposed to be utilized/ re-financing, and where appropriate, allocated for refinancing, the also clarify which investments or following details shall be project portfolios may be refinanced, disclosed with regard to such and, to the extent relevant, the refinancing: expected look-back period for i. Details of projects/portfolios refinanced eligible green projects. That are proposed to be - in line with recommendations of refinanced. Gbp 2021, the disclosure of ii. Percent share of financing utilization of proceeds of green debt and refinancing. Page 10 of 15 11 securities is proposed to be aligned iii. Look-back period for with gbp principles for better refinanced green projects. Disclosure on use of proceeds for explanation: look-back refinancing. Period in the clause above refers to a maximum period in the past that an issuer will look back to identify assets/earlier disbursements to such eligible green projects that will be included in the disclosures associated with the green debt securities issued. F. Disclosure the extant sebi framework does not in addition to the existing requirements provide for specific disclosure disclosure requirements in the clause 1 of regarding identification and offer documents, the issuer chapter ix of management of perceived social and may provide information on the sebi environmental risks associated with processes by which the issuer circular dated the projects to be financed through identifies and manages december 17, gds. Perceived social and 2021 environmental risks gbp 2021 recommends, the issuer associated with the project(s) should provide complementary proposed to be financed/ information on processes by which refinanced. The issuer identifies and manages perceived social and environmental risks associated with the relevant project(s). In line with recommendations of gbp 2021, the issuer may be required to disclose the processes by which the issuer identifies and manages perceived social and environmental risks. G. Disclosure the extant sebi framework provides in order to enhance disclosure requirements that an issuer of gds may appoint an requirements for third party clause 1 of independent third party reviewer/ review and certification of chapter ix of certifier, for reviewing/ certifying the projects and processes with the sebi processes including project regard to issues of gds, circular dated evaluation and selection criteria, clause 1.5 of chapter ix of december 17, project categories eligible for the sebi circular dated 2021 financing by green debt securities, december 17, 2021 may be etc. Such appointment is at the option amended as follows: page 11 of 15 12 of the issuer; however, any such appointment of third party appointment of reviewer/ certifier, is reviewers/ certifiers/ required to be disclosed in the offer auditors: document. In this regard, gbp 2021 an issuer may appoint recommends the following: external reviewers/ certifiers/ - issuers should appoint (an) external auditors for purposes review provider(s) to assess through including, but not limited to, (i) a pre-issuance external review the assessment of alignment of alignment of their green bond or the objectives of the green green bond programme and/ or bond to be issued with the framework with the four core clause xx of this circular (ii) components of the gbp. Post-issue management of use of proceeds from the - post issuance, it is recommended green bond (iii) verification of that an issuers management of the internal tracking and the proceeds be supplemented by the fund allocation from the green use of an external auditor, or other bond proceeds to eligible third party, to verify the internal green projects etc. Tracking and the allocation of funds from the green bond proceeds to such appointment is optional eligible green projects. For the issuer. However, any such appointment of reviewer/ - the gbp encourage external review certifier/ auditor shall be providers to disclose their credentials disclosed in the offer and relevant expertise and document(s). Such external communicate clearly the scope of the reviews shall also be review(s) conducted. Issuers should disclosed on the stock make external reviews publicly exchanges where the bonds available on their website and/or are listed or proposed to be through any other accessible listed. Communication channel as appropriate and if feasible, as well as use the template for external reviews available in the sustainable finance section of icmas website. In line with the recommendations of gbp 2021, the disclosure requirement may be amended to disclose the aforementioned information. H. Disclosure the extant sebi framework provides in addition to the existing requirements that an issuer of green debt securities disclosure requirements for page 12 of 15 13 clause 2 of or any agent appointed by the issuer measurement of impact chapter ix of complying with globally accepted reporting, the following may the sebi standard(s) for the issuance of green also be included: circular dated debt securities including a. Impact reporting: december 17, measurement of the environmental 2021. Impact, identification of the project(s) an issuer, or any agent and/ or asset(s), utilisation of appointed by the issuer shall proceeds, etc. , shall disclose the disclose information same in the offer document and/ pertaining to reporting of the or as part of continuous disclosures. Environmental impact of the projects financed by the green with regard to impact reporting, the debt securities in the offer gbp 2021 recommends the document and/or as a following: continuous disclosure. Reporting standards or - the annual report should include a taxonomies followed by the list of the projects to which green issuer with regard to reporting bond proceeds have been allocated, of environmental impact may as well as a brief description of the also be disclosed, if any. Projects, the amounts allocated, and such disclosure is optional their expected impact. And is at the part of issuer and may be done on a project-by- in line with recommendations of gbp project basis. 2021, the disclosure requirements may be amended to provide for impact reporting as such disclosure will enable investors to gather information pertaining to the impact of the project on the environment. 10.1 views/ comments sought on: a. Is there a need to revisit the existing green bond framework on the lines of the gbps mentioned above? If so, which are the recommendations that have to be adopted? B. Please offer comments on whether any additional policies/ schemes, if any. 11. Greenwashing 11.1. One of the main hurdles for further growth has been a consistent and robust approach to identifying what is considered green. A lack of clarity in this regard leads greenwashing which is defined as the practice of channelling proceeds from green bonds towards projects or activities having negligible or negative environmental benefits. Various issuers page 13 of 15 14 in the past have been accused of greenwashing, and the same has caused significant reputational risk for socially conscious investors seeking to diversify their investment portfolios by investing in environmental, social and governance (esg) practices. 11.2. Bankers and lenders are cautious about financing new asset classes in the green bond market in the absence of adherence to widely accepted standards and taxonomies. On the other hand, no issuer would want to risk their reputation by issuing a bond which is criticized for not being adequately green or transformational, particularly due to lack of availability of a domestic green assessment framework to rate them in a transparent and uniform manner. 11.3. Sebi guidelines on green bonds provide a list of categories wherein funds raised through debt securities are eligible to be classified as green debt securities. 11.4. While china and the european union (eu) have issued their own taxonomies regarding assets eligible to be financed through green bonds, india does not have a detailed taxonomy of such assets. Further, in the absence of the same, major issuers such as sbi, yes bank, ireda etc. Have their own internal green bond frameworks to identify and finance projects. 11.5. In this regard, it is proposed that the list of sectors identified as eligible under the sebi framework may be updated in line with the updated icma guidelines and market feedback/public consultations. 11.6. As per extant regulations, green bonds which do not permit recourse to the issuer are not available in india. However, green revenue bonds or green project bonds as defined by icma are not envisaged under extant sebi regulations. 11.7. Similarly, securitized debt obligations as defined under sebi (issue and listing of securitized debt instruments and security receipts) regulations, 2008 that may have green assets in their underlying asset pool are also excluded from the current framework. 11.8. Feedback is sought regarding addition of such alternative debt securities as defined by icma to be considered as eligible green debt securities. An indicative description of the same is as follows: a. Green revenue bonds which limit exposure to the green project being financed and does not provide recourse to the issuer may b. Green project bond which provides for credit exposure to a single / multiple green projects, with or without recourse to the issuer c. Green securitized bonds collateralized by cash flows from the projects. 11.9. Views/ comments sought on: a. Whether such sub-categorization of green bonds will help reduce greenwashing? B. Please offer comments on whether any additional regulatory/ technical solutions against greenwashing? Page 14 of 15 15 12. Public comments on this consultation paper public comments are invited on the proposal. The comments/ suggestions may be provided as per the format given below: name of the person/entity proposing comments: name of the organization (if applicable): contact details: category: whether market intermediary/ participant (mention type/ category) or public (investor, academician etc. ) sr. Extract from issues (with page/para nos. , if proposals/ rationale no. Consultation paper applicable) suggestions kindly mention the subject of the communication as, comments on consultation paper on green and blue bonds as a mode of sustainable finance. Comments as per aforesaid format may be sent to the following, latest by august 31, 2022, in any of the following manner: a. By email to: pradeepr@sebi. Gov. In; nikhilc@sebi. Gov. In; and vikramj@sebi. Gov. In; or b. By post to the following address: pradeep ramakrishnan, general manager, department of debt and hybrid securities, securities and exchange board of india, sebi bhavan, c4-a, g-block, bandra kurla complex, bandra (east), mumbai - 400051 issued on: august 04, 2022 page 15 of 15"}

In [30]:
predictor.predict(input)

[{'summary_text': 'The World Bank today announced that it has issued a draft green bond policy that includes a provision'}]

In [31]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-led-base-16384")  
model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-led-base-16384")

padding = "max_length" 

#text=""" manager in connection with manipulative trading in the securities of HD View 360 Inc., a U.S.-based microcap issuer.  The SEC also announced charges against HD View's CEO, another individual, and three entities they control for manipulating HD View's securities as well as the securities of another microcap issuer, West Coast Ventures Group Corp.  The SEC further announced the institution of an order suspending trading in the securities of HD View.These charges arise in part from an undercover operation by the Federal Bureau of Investigation, which also resulted in related criminal prosecutions against these defendants by the Office of the United States Attorney for the Eastern District of New York.In a complaint filed in the U.S. District Court for the Eastern District of New York, the SEC alleges that Beaufort Securities Ltd. and Peter Kyriacou, an investment manager at Beaufort, manipulated the market for HD View's common stock.  The scheme involved an undercover FBI agent who described his business as manipulating U.S. stocks through pump-and-dump schemes.  Kyriacou and the agent discussed depositing large blocks of microcap stock in Beaufort accounts, driving up the price of the stock through promotions, manipulating the stock's price and volume through matched trades, and then selling the shares for a large profit.The SEC's complaint against Beaufort and Kyriacou alleges that they:opened brokerage accounts for the undercover agent in the names of nominees in order to conceal his identity and his connection to the anticipated trading activity in the accounts suggested that the undercover agent could create the false appearance that HD View's stock was liquid in advance of a pump-and-dump by "gam[ing] the market" through matched trades executed multiple purchase orders of HD View shares with the understanding that Beaufort's client had arranged for an associate to simultaneously offer an equivalent number of shares at the same priceA second complaint filed by the SEC in the U.S. District Court for the Eastern District of New York alleges that in a series of recorded telephone conversations with the undercover agent, HD View CEO Dennis Mancino and William T. Hirschy agreed to manipulate HD View's common stock by using the agent's network of brokers to generate fraudulent retail demand for the stock in exchange for a kickback from the trading proceeds.  According to the complaint, the three men agreed that Mancino and Hirschy would manipulate HD View stock to a higher price before using the agent's brokers to liquidate their positions at an artificially inflated price.  The SEC's complaint also alleges that Mancino and Hirschy executed a "test trade" on Jan. 31, 2018, coordinated by the agent, consisting of a sell order placed by the defendants filled by an opposing purchase order placed by a broker into an account at Beaufort.  Unbeknownst to Mancino and Hirschy, the Beaufort account used for this trade was a nominal account that was opened and funded by the agent.  The SEC's complaint also alleges that, prior to their contact with the undercover agent, Mancino and Hirschy manipulated the market for HD View and for West Coast by using brokerage accounts that they owned, controlled, or were associated with –including TJM Investments Inc., DJK Investments 10 Inc., WT Consulting Group LLC – to effect manipulative "matched trades."The SEC's complaint against Beaufort and Kyriacou charges the defendants with violating Section 10(b) of the Securities Exchange Act of 1934 and Rule 10b-5 thereunder.  The SEC also charged Hirschy, Mancino, and their corporate entities with violating Section 17(a)(1) of the Securities Act of 1933, Sections 9(a)(1), 9(a)(2), and 10(b) of the Exchange Act and Rules 10b-5(a) and (c) thereunder.  The SEC is seeking injunctions, disgorgement, prejudgment interest, penalties, and penny stock bars from Beaufort and Kyriacou.  With respect to Hirschy, Mancino, and their corporate entities, the SEC is seeking injunctions, disgorgement, prejudgment interest, penalties, penny stock bars, and an officer-and-director bar against Mancino.The investigation was conducted in the SEC's New York Regional Office by Tejal Shah and Joseph Darragh, Lorraine Collazo, and Michael D. Paley of the Microcap Fraud Task Force and supervised by Lara S. Mehraban, and in Washington, D.C. by Patrick L. Feeney, Robert Nesbitt, and Kevin Guerrero, and supervised by Antonia Chion.  Preethi Krishnamurthy and Ms. Shah will lead the SEC's litigation against Beaufort and Kyriacou.  Ann H. Petalas and Mr. Feeney, under the supervision of Cheryl Crumpton, will handle the SEC's litigation against Mancino, Hirschy, and their entities.  The SEC appreciates the assistance of the Office of the United States Attorney for the Eastern District of New York, the Federal Bureau of Investigation, the Internal Revenue Service, the Alberta Securities Commission, the Ontario Securities Commission, the Financial Conduct Authority of the United Kingdom, and the Financial Industry Regulatory Authority.The Commission's investigation in this matter is continuing."""
text="1 consultation paper on green and blue bonds as a mode of sustainable finance 1. Background 1.1 sustainable finance is the process of taking due account of environmental, social and governance (esg) considerations while making investment decisions in the financial sector, leading to increased longer-term investments into sustainable economic activities and projects.1 1.2 the need for sustainable finance was felt in the aftermath of repeated environmental disasters and extreme weather events, corporate finance being driven purely by profit motives and corporate governance norms being followed as the exception rather than the norm. This has been bolstered by the demand for sustainable avenues of investments from a section of the new- age investors as well as with the substantial rise in the prominence of the stakeholders theory over shareholders theory. 1.3 the cop 26 mandate: in the 2021 united nations climate change conference, more commonly referred to as cop26, held in glasgow, scotland, 197 countries, including india, have made enhanced commitments towards mitigating climate change and promising more climate finance for developing countries to adapt to climate impacts. This means that sustainable finance will be the mainstay of world business, which in turn, will mean more demand for funds in this domain. 2. Sebis regulatory regime on green bonds: 2.1 the concept of green debt securities was introduced under the erstwhile securities and exchange board of india (issue and listing of debt securities) regulations, 2008 vide circular dated may 30, 20172. 2.2 regulation 2(1)(q) of the securities and exchange board of india (issue and listing of non- convertible securities) regulations (ncs regulations) defines green debt security, which reads as under: green debt security means a debt security issued for raising funds that are to be utilised for project(s) and/or asset(s) falling under any of the following categories, subject to the conditions as may be specified by the board from time to time: (i) renewable and sustainable energy including wind, solar, bioenergy, other sources of energy which use clean technology, (ii) clean transportation including mass/public transportation, (iii) sustainable water management including clean and/or drinking water, water recycling, 1 https: //www. Worldbank. Org/en/topic/financialsector/brief/sustainable-finance 2 subsumed in the operational circular on the ncs regulations, 2021 please see chapter ix page 1 of 15 2 (iv) climate change adaptation, (v) energy efficiency including efficient and green buildings, (vi) sustainable waste management including recycling, waste to energy, efficient disposal of wastage, (vii) sustainable land use including sustainable forestry and agriculture, afforestation, (viii) biodiversity conservation, or (ix) a category as may be specified by the board, from time to time 2.3 further, chapter ix of the operational circular on the ncs regulations, 2021 dated august 10, 2021, as amended on date, specifies the following with reference to issuers of green debt securities: a. Additional disclosure requirements in the offer document; b. Continuous disclosure requirements in annual report and financial results; c. Responsibilities of the issuer. 3. Mandate of this consultation paper 3.1. Since the framework of green debt securities was laid down by sebi, there have been multiple events in the sustainable finance space around the world, thereby necessitating a review in the indian context. 3.2. In this context, sebi, through this consultation paper, is seeking public comments on a proposed regulatory framework: a. To amplify the definition of green debt securities, b. To introduce the concept of blue bonds c. To reduce the compliance cost for issuers of green debt securities with while not creating any perverse incentives that may lead to greenwashing. 3.3. Suggestions are also solicited towards increasing avenues for sustainable finance in india, while considering indias unique goals of pursuing high growth with sustainable development. 3.4. This consultation paper follows a series of discussions held with multiple stakeholders, including the sub-committee formed under the corporate bonds securitization advisory committee (cobosac) of sebi. 4. Global scenario 4.1 issuance of green, social, sustainable and sustainability-linked (gsss) bonds as a percentage of total global bond issuance rose from roughly 2 per cent at the start of 2018 to a peak of over 12 per cent at the end of 20213. Prices continued to rise as the demand from investors exploded. Figures from data provider epfr, as given in the graph below, show that flows into esg and socially responsible investing bond funds rose from a net of $4.2bn in 2018 to $102bn in 2021. 3 https: //www. Ft. Com/content/32dbf37c-8ff5-436b-88f3-9873fc864a7b page 2 of 15 3 4.2 with economic growth slowing globally owing to high inflation, high commodity prices and higher interest rates issuers of bonds, both green and conventional, have pulled back. As the sustainable bonds are subject to the same headwinds as that of the broader market, green bond issuances in this year are expected to be roughly in line with 2021. That is a notable slowdown, given how quickly the market has been growing. Despite the above fact, rating agencies forecast that global issuance of gsss bonds will be around $1tn this year4. 5. Present fund raising through green bonds india 5.1. As per the economic survey 2019-205, climate bonds worth usd 10.3 billion were issued in the first half of 2019 in india. Climate bonds in india are focused on green bonds, which are specifically linked to climate-change mitigation, adaptation and resilience. 5.2. A number of government agencies have contributed to issuance of such bonds, particularly indian renewable energy development agency (ireda) and the indian railway finance corporation (irfc). 5.3. Currently, green bonds raised by various issuers are listed on india inx. Similarly, various issuers have also issued and listed their bonds on other international stock exchanges such as luxembourg se, singapore exchange, etc. 5.4. Indian companies raised nearly usd 7 billion through esg and green bonds in 2021, compared to usd 1.4 billion and usd 4 billion in 2020 and 2019, respectively. 5.5. As of february 12, 2020, the outstanding amount of green bonds in india was us$16.3 billion. 4 https: //www. Climatebonds. Net/2022/01/500bn-green-issuance-2021-social-and-sustainable-acceleration-annual-green-1tn- sight-market 5 https: //www. Indiabudget. Gov. In/budget2020-21/economicsurvey/index. Php page 3 of 15 4 5.6. As per economic survey 2020-21 and emerging markets green bonds report 2020, india occupies the 2nd spot (after china) in the cumulative emerging market green bond issuance, 2012-20206. Cumulative emerging market green bond issuance, 2012-2020 (us$ million) 5.7. The above graph shows that in most countries, green bond issuances by corporate entities vastly exceed the issuances by government/ sovereign agencies. The only exceptions are chile, poland, indonesia, hungary, egypt and argentina. 5.8. As on june 30, 2022, under the sebi framework for green bonds, 14 issuers have raised rs. 4,539 crore (about $ 0.5 billion). 6 https: //www. Indiabudget. Gov. In/budget2021-22/economicsurvey/index. Php page 4 of 15 5 5.9. As can be seen from above, there is a stark divergence between the amount raised by listing green bonds in indian jurisdictions versus foreign jurisdictions or autonomous jurisdictions like inx (india international exchange). It may be noted that issuers are finding it more attractive to list on bourses falling outside sebi framework. 5.10. Most of the green bonds issued by indian issuers are listed on offshore exchanges. Informal consultation with the industry participants attribute this to: - lack of demand for such bonds among the indian investors - favourable pricing in overseas market. 6. Scope for sustainable finance in india 6.1 at the cop-26 summit in glasgow, india had made the following commitments towards its climate change goals7: a. Raising non-fossil fuel based energy capacity to 500 gw. B. Lowering total projected carbon emission by one billion tons. C. Meeting 50% of the countrys energy needs through renewable sources. D. Reduce the carbon intensity of the economy to sub 45% level. E. Commitment to achieve net-zero emissions by 2070. 6.2 government of india has initiated the following schemes/ policies in pursuit of the goals towards cop-26 targets: a. Commission on air quality management (caqm) - phasing out of coal use in national capital region from january 2023, but exempting thermal power plants using low- sulphur coal. B. National biomass co-firing policy to use biomass like paddy stubble and cow-dung in coal power plants8. C. Mandate the use of flue gas desulphurization (fgd) techniques in both existing and upcoming thermal power plants. D. Government has targeted to achieve 100 million tonnes of coal gasification by 20309. E. National mission on transformative mobility and battery storage10. F. Productivity linked incentives (pli) scheme on advanced cell chemistry and battery storage11. G. India cooling action plan12. 7 https: //www. Mea. Gov. In/speeches- statements. Htm? Dtl/34466/national_statement_by_prime_minister_shri_narendra_modi_at_cop26_summit_in_glasgow 8 https: //powermin. Gov. In/sites/default/files/revised_biomass_policy_dtd_08102021. Pdf 9 https: //pib. Gov. In/pressreleaseshare. Aspx? Prid=1650096 10 https: //www. Niti. Gov. In/e-mobility-national-mission-transformative-mobility-and-battery-storage 11 https: //www. Niti. Gov. In/sites/default/files/2022-02/need-for-acc-energy-storage-in-india. Pdf 12 https: //www. Iea. Org/policies/7455-india-cooling-action-plan-icap page 5 of 15 6 6.3 views/ comments sought on: a. Whether the above mentioned initiatives offer any scope for financing through green bonds? B. Please offer comments on whether there are any additional policies/ schemes in line with the above objective, if any? 7. Potential in emerging areas 7.1. The sub-committee constituted by the cobosac had presented its report, which has covered areas like priority sector lending to green sectors, investment by institutional investors, tax recommendation for green bonds and updation of extant guidelines in line with updated green bond principles published by the international capital markets association13. 7.2. There is a need to further channel the investment in emerging areas like data centers which are receiving policy traction (draft data center policy). This field assumes importance in light of government push for data localization and the storage necessities for large volumes of data collected in india. Data centers need tremendous cooling facilities, some of which can be attractive avenues for sustainable finance. 7.3. New concepts like circular economy and regional resource based manufacturing are gaining currency, as a mode to balance job creation with inter-generational environmental equity. Though definitions of these need deeper analysis, there is a need for encouraging flow of green finance in these initiatives. The government of india has presented schemes like one district one product, zero effect zero defect and promotion of these goods through local post offices and railway stations. 7.4. Ecosystem services evaluation- the natural capital accounting and valuation of ecosystem services (ncaves) india forum is being organized by ministry of statistics and programme implementation (mospi) in collaboration with the united nations statistics division (unsd), european union and un environment. India has also implemented the system of environmental economic accounting (seea) which provides policy makers and decision-makers with information that accounts for the value nature contributes to our economies.14 7.5. Agritech companies investing in net-zero for agriculture: - agriculture and allied sectors like animal husbandry are responsible for over 85% of anthropogenic greenhouse gas emissions. Sustainable finance and technology in this sector can take out a major source of climate change. 7.6. National hydrogen mission- india is targeting to produce three-fourths of its hydrogen from renewable resources by 2050.15 7.7. Blue economy and ocean conservation.16 13 https: //www. Icmagroup. Org/sustainable-finance/the-principles-guidelines-and-handbooks/green-bond-principles-gbp/ 14 https: //seea. Un. Org/content/natural-capital-accounting-and-valuation-ecosystem-services-india 15 https: //pib. Gov. In/pressreleasepage. Aspx? Prid=1799067 16 https: //www. Worldbank. Org/en/news/infographic/2017/06/06/blue-economy page 6 of 15 7 8. Potential for blue bonds 8.1. According to the world bank, the blue economy is the sustainable use of ocean resources for economic growth, improved livelihoods, and jobs while preserving the health of ocean ecosystem. 17 european commission defines it as all economic activities related to oceans, seas and coasts. 8.2. For a country like india that has a 7500 km long coastline and 14,500 km of navigable inland waterways, the development of blue economy can serve as a growth catalyst in realizing the vision to become a $10 trillion economy by 2032. The blue economy currently comprises 4.1% of indias economy. 8.3. The government of india has kickstarted the initiative to develop the blue economy by announcing project sagarmala, a first of its kind port-led development programme in india. The sagarmala programme plans to bring in development around india's coastal regions based on the following four pillars: port modernization & new port development, port connectivity enhancement & port-linked industrialization. 8.4. The ministry of earth sciences (moes) has rolled out the draft blue economy policy, inviting suggestions and inputs from various stakeholders. 8.5. It emphasizes policies across several key sectors to achieve holistic growth of indias economy. It recognizes the following thematic areas: a. National accounting framework for the blue economy and ocean governance. B. Coastal marine spatial planning and tourism. C. Marine fisheries, aquaculture, and fish processing. D. Manufacturing, emerging industries, trade, technology, services, and skill development. E. Logistics, infrastructure and shipping, including trans-shipments. F. Coastal and deep-sea mining and offshore energy. G. Security, strategic dimensions, and international engagement. H. Indias exclusive economic zone of over 2 million square kilometres has a huge living and non-living resources with significant recoverable resources such as crude oil and natural gas. 9. Scope of blue bonds in india india has tremendous scope for deployment of blue bonds in various aspects of the blue economy, like: 9.1 oceanic resource mining: polymetallic nodules and methane hydrates are abundant in the indian ocean region. Mining of polymetallic nodules present in the seabed in the central indian ocean basin can help india improve availability of nickel, copper, cobalt and manganese. Through an agreement with the international seabed authority, india has a right to explore and mine polymetallic nodules over 750,000 square km.18 17 https: //www. Worldbank. Org/en/news/feature/2018/10/29/sovereign-blue-bond-issuance-frequently-asked- questions 18 https: //pib. Gov. In/newsite/printrelease. Aspx? Relid=170138 page 7 of 15 8 9.2 sustainable fishing: as the developing countries face new challenges at wto to phase out illegal, unreported and unregulated (iuu) fishing, it is time the country phases out exploitative fishing devices like bottom trawlers. India has an exclusive economic zone of 2.37 million sq. Km. 9.3 national offshore wind energy policy: ministry has set a target of 5.0 gw of offshore wind installations by 2022 and 30 gw by 2030 which has been issued to give confidence to the project developers in indian market. 9.4 coral degradation: india can invest in technologies like biorock to rejuvenate the corals along its coastline, by forming an artificial substrate. 9.5 india has acceded to the marine pollution (marpol) convention covering prevention of pollution of the marine environment by ships from operational or accidental causes. 9.6 geoengineering techniques like ocean fertilization or ocean nourishment is a type of climate engineering based on the purposeful introduction of plant nutrients to the upper ocean to increase marine food production and to remove carbon dioxide from the atmosphere. 9.7 blue flag beach is an eco-tourism model endeavouring to provide the tourists/ beachgoers clean and hygienic bathing water, facilities, a safe and healthy environment and sustainable development of the area.19 9.8 views/ comments sought on: a. Whether the above mentioned initiatives offer any scope for financing through blue bonds? B. Whether introducing coloured bonds (blue bonds for blue economy, yellow bonds for solar power) will help increase channels for funding to green projects? C. Please offer comments on whether any additional policies/ schemes, if any? 10. Recommendations made by the sub-committee of cobosac on green bonds the sub-committee has proposed the following amendments to sebi guidelines in order to align with the updated green bond principles (gbp) published by the international capital market association (icma): sr. Existing need for change proposed amendment no. Clause a. Definition of the extant sebi framework defines the following may be included green debt gds as debt security issued for in addition to the existing list security (gds) raising funds that are to be utilised for of eligible categories for regulation projects or assets falling under issuance of gds: 2(1)(q) of sebi certain categories. A. Pollution prevention and (issue and gbp 2021 have included two control (including reduction of listing of non- additional categories, namely, air emissions, greenhouse convertible pollution prevention and control and gas control, soil remediation, securities), waste 19 http: //blueflagindia. Org/ page 8 of 15 9 regulations, circular economy adapted products, prevention/minimization, 2021) as eligible green projects. Waste reduction, waste in order to align with gbp 2021 recycling and energy/ principles, these additional emission efficient waste to categories may be included. Energy). B. Circular economy adapted products, production technologies and processes (such as the design and introduction of reusable, recyclable and refurbished materials, components and products, circular tools and services) and/or certified eco efficient products. B. Utilisation of the extant sebi framework for gds in addition to the existing issue do not mandate disclosure of disclosure requirements, we proceeds temporary placement for the balance may include that, the issuer clause 1 of of unallocated net proceeds, if any. Shall disclose the intended chapter ix of gbp 2021 have recommended that types of temporary placement sebi circular the issuer should make known to for the balance of unallocated dated investors the intended types of net proceeds. December 17, temporary placement for the balance 2021 of unallocated net proceeds. In view of the same, the temporary utilization of proceeds raised for green debt securities may be specified in order to have better disclosure and increased transparency. C. Utilisation of the extant sebi framework for gds in addition to the existing issue do not mandate disclosure of disclosure requirements, we proceeds utilization of issue proceeds for each may include that, the clause 2 of debt security issued by an issuer. Utilisation of proceeds from chapter ix of gbp 2021 recommends that the each issue of gds made by sebi circular proceeds of green bonds can be an issuer shall be tracked and dated managed per bond (bond-by-bond disclosed separately (i. E. On a december 17, approach) or on an aggregated basis bond by bond approach) or on 2021 for multiple green bonds (portfolio an aggregated basis for approach). Multiple green bonds in order to have clear distinction (portfolio approach). Between the uses of proceeds of multiple issuances of green debt security by a single issuer for various page 9 of 15 10 green projects, bond by bond approach may be adopted. D. Disclosure the extant sebi framework for gds in addition to the existing requirements does not mandate disclosure of disclosure requirements for clause 2 of taxonomy, standards, certifications, selection of projects to be chapter ix of etc. Referred for project selection by financed through gds, we the sebi the issuer. May include that, the issuer circular dated gbp 2021 recommends the issuer to shall disclose any december 17, provide information, if relevant, on the taxonomies, green standards 2021 alignment of projects with official or or certifications, if referenced market-based taxonomies, related in the project selection. Eligibility criteria, including if issuers should provide applicable, exclusion criteria; and information regarding the also disclose any green standards or alignment of projects with said certifications referenced in project taxonomies, related eligibility selection. Criteria, including if applicable, in line with recommendations of gbp exclusion criteria. 2021, the aforementioned disclosure may be made applicable to issuers of gds. E. Disclosure with respect to refinancing of assets in order to enhance disclosure requirements from proceeds of gds, the extant requirements for refinancing clause 1 of sebi framework requires the issuer of projects through gds, chapter ix of to disclose details of projects and/ or clause 1.4 of chapter ix of the sebi assets or areas where the issuer the sebi circular dated circular dated proposed to utilize the proceeds of december 17, 2021 may be december 17, the issue of gds including towards amended as follows: 2021 refinancing of existing green projects a. Details of identified and/or assets, if any. Project(s) and/or asset(s) gbp 2021 recommends the where the issuer proposes to following: utilise/ allocate the proceeds - in the event that all or a proportion of the issue of gds. Of the proceeds are or may be used b. In case a portion of the for refinancing, issuers provide an proceeds of the issue are estimate of the share of financing vs. Proposed to be utilized/ re-financing, and where appropriate, allocated for refinancing, the also clarify which investments or following details shall be project portfolios may be refinanced, disclosed with regard to such and, to the extent relevant, the refinancing: expected look-back period for i. Details of projects/portfolios refinanced eligible green projects. That are proposed to be - in line with recommendations of refinanced. Gbp 2021, the disclosure of ii. Percent share of financing utilization of proceeds of green debt and refinancing. Page 10 of 15 11 securities is proposed to be aligned iii. Look-back period for with gbp principles for better refinanced green projects. Disclosure on use of proceeds for explanation: look-back refinancing. Period in the clause above refers to a maximum period in the past that an issuer will look back to identify assets/earlier disbursements to such eligible green projects that will be included in the disclosures associated with the green debt securities issued. F. Disclosure the extant sebi framework does not in addition to the existing requirements provide for specific disclosure disclosure requirements in the clause 1 of regarding identification and offer documents, the issuer chapter ix of management of perceived social and may provide information on the sebi environmental risks associated with processes by which the issuer circular dated the projects to be financed through identifies and manages december 17, gds. Perceived social and 2021 environmental risks gbp 2021 recommends, the issuer associated with the project(s) should provide complementary proposed to be financed/ information on processes by which refinanced. The issuer identifies and manages perceived social and environmental risks associated with the relevant project(s). In line with recommendations of gbp 2021, the issuer may be required to disclose the processes by which the issuer identifies and manages perceived social and environmental risks. G. Disclosure the extant sebi framework provides in order to enhance disclosure requirements that an issuer of gds may appoint an requirements for third party clause 1 of independent third party reviewer/ review and certification of chapter ix of certifier, for reviewing/ certifying the projects and processes with the sebi processes including project regard to issues of gds, circular dated evaluation and selection criteria, clause 1.5 of chapter ix of december 17, project categories eligible for the sebi circular dated 2021 financing by green debt securities, december 17, 2021 may be etc. Such appointment is at the option amended as follows: page 11 of 15 12 of the issuer; however, any such appointment of third party appointment of reviewer/ certifier, is reviewers/ certifiers/ required to be disclosed in the offer auditors: document. In this regard, gbp 2021 an issuer may appoint recommends the following: external reviewers/ certifiers/ - issuers should appoint (an) external auditors for purposes review provider(s) to assess through including, but not limited to, (i) a pre-issuance external review the assessment of alignment of alignment of their green bond or the objectives of the green green bond programme and/ or bond to be issued with the framework with the four core clause xx of this circular (ii) components of the gbp. Post-issue management of use of proceeds from the - post issuance, it is recommended green bond (iii) verification of that an issuers management of the internal tracking and the proceeds be supplemented by the fund allocation from the green use of an external auditor, or other bond proceeds to eligible third party, to verify the internal green projects etc. Tracking and the allocation of funds from the green bond proceeds to such appointment is optional eligible green projects. For the issuer. However, any such appointment of reviewer/ - the gbp encourage external review certifier/ auditor shall be providers to disclose their credentials disclosed in the offer and relevant expertise and document(s). Such external communicate clearly the scope of the reviews shall also be review(s) conducted. Issuers should disclosed on the stock make external reviews publicly exchanges where the bonds available on their website and/or are listed or proposed to be through any other accessible listed. Communication channel as appropriate and if feasible, as well as use the template for external reviews available in the sustainable finance section of icmas website. In line with the recommendations of gbp 2021, the disclosure requirement may be amended to disclose the aforementioned information. H. Disclosure the extant sebi framework provides in addition to the existing requirements that an issuer of green debt securities disclosure requirements for page 12 of 15 13 clause 2 of or any agent appointed by the issuer measurement of impact chapter ix of complying with globally accepted reporting, the following may the sebi standard(s) for the issuance of green also be included: circular dated debt securities including a. Impact reporting: december 17, measurement of the environmental 2021. Impact, identification of the project(s) an issuer, or any agent and/ or asset(s), utilisation of appointed by the issuer shall proceeds, etc. , shall disclose the disclose information same in the offer document and/ pertaining to reporting of the or as part of continuous disclosures. Environmental impact of the projects financed by the green with regard to impact reporting, the debt securities in the offer gbp 2021 recommends the document and/or as a following: continuous disclosure. Reporting standards or - the annual report should include a taxonomies followed by the list of the projects to which green issuer with regard to reporting bond proceeds have been allocated, of environmental impact may as well as a brief description of the also be disclosed, if any. Projects, the amounts allocated, and such disclosure is optional their expected impact. And is at the part of issuer and may be done on a project-by- in line with recommendations of gbp project basis. 2021, the disclosure requirements may be amended to provide for impact reporting as such disclosure will enable investors to gather information pertaining to the impact of the project on the environment. 10.1 views/ comments sought on: a. Is there a need to revisit the existing green bond framework on the lines of the gbps mentioned above? If so, which are the recommendations that have to be adopted? B. Please offer comments on whether any additional policies/ schemes, if any. 11. Greenwashing 11.1. One of the main hurdles for further growth has been a consistent and robust approach to identifying what is considered green. A lack of clarity in this regard leads greenwashing which is defined as the practice of channelling proceeds from green bonds towards projects or activities having negligible or negative environmental benefits. Various issuers page 13 of 15 14 in the past have been accused of greenwashing, and the same has caused significant reputational risk for socially conscious investors seeking to diversify their investment portfolios by investing in environmental, social and governance (esg) practices. 11.2. Bankers and lenders are cautious about financing new asset classes in the green bond market in the absence of adherence to widely accepted standards and taxonomies. On the other hand, no issuer would want to risk their reputation by issuing a bond which is criticized for not being adequately green or transformational, particularly due to lack of availability of a domestic green assessment framework to rate them in a transparent and uniform manner. 11.3. Sebi guidelines on green bonds provide a list of categories wherein funds raised through debt securities are eligible to be classified as green debt securities. 11.4. While china and the european union (eu) have issued their own taxonomies regarding assets eligible to be financed through green bonds, india does not have a detailed taxonomy of such assets. Further, in the absence of the same, major issuers such as sbi, yes bank, ireda etc. Have their own internal green bond frameworks to identify and finance projects. 11.5. In this regard, it is proposed that the list of sectors identified as eligible under the sebi framework may be updated in line with the updated icma guidelines and market feedback/public consultations. 11.6. As per extant regulations, green bonds which do not permit recourse to the issuer are not available in india. However, green revenue bonds or green project bonds as defined by icma are not envisaged under extant sebi regulations. 11.7. Similarly, securitized debt obligations as defined under sebi (issue and listing of securitized debt instruments and security receipts) regulations, 2008 that may have green assets in their underlying asset pool are also excluded from the current framework. 11.8. Feedback is sought regarding addition of such alternative debt securities as defined by icma to be considered as eligible green debt securities. An indicative description of the same is as follows: a. Green revenue bonds which limit exposure to the green project being financed and does not provide recourse to the issuer may b. Green project bond which provides for credit exposure to a single / multiple green projects, with or without recourse to the issuer c. Green securitized bonds collateralized by cash flows from the projects. 11.9. Views/ comments sought on: a. Whether such sub-categorization of green bonds will help reduce greenwashing? B. Please offer comments on whether any additional regulatory/ technical solutions against greenwashing? Page 14 of 15 15 12. Public comments on this consultation paper public comments are invited on the proposal. The comments/ suggestions may be provided as per the format given below: name of the person/entity proposing comments: name of the organization (if applicable): contact details: category: whether market intermediary/ participant (mention type/ category) or public (investor, academician etc. ) sr. Extract from issues (with page/para nos. , if proposals/ rationale no. Consultation paper applicable) suggestions kindly mention the subject of the communication as, comments on consultation paper on green and blue bonds as a mode of sustainable finance. Comments as per aforesaid format may be sent to the following, latest by august 31, 2022, in any of the following manner: a. By email to: pradeepr@sebi. Gov. In; nikhilc@sebi. Gov. In; and vikramj@sebi. Gov. In; or b. By post to the following address: pradeep ramakrishnan, general manager, department of debt and hybrid securities, securities and exchange board of india, sebi bhavan, c4-a, g-block, bandra kurla complex, bandra (east), mumbai - 400051 issued on: august 04, 2022 page 15 of 15"
input_tokenized = tokenizer.encode(text, return_tensors='pt',padding=padding,pad_to_max_length=True, max_length=6144,truncation=True)
summary_ids = model.generate(input_tokenized,
                                  num_beams=4,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2,
                                  min_length=350,
                                  max_length=500)
summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
### Summary Output

# On March 2, 2018, the Securities and Exchange Commission charged Beaufort Securities Ltd. and Peter Kyriacou, an investment manager at Beaufort, with manipulating the market for HD View 360 Inc., a U.S.-based microcap issuer.  The SEC also announced charges against HD View's CEO, another individual, and three entities they control for manipulating HD View through pump-and-dump schemes.  According to the SEC's complaint, the defendants discussed depositing large blocks of microcap stock in Beaufort accounts, driving up the price of the stock through promotions, manipulating the stock's price and volume through matched trades, and then selling the shares for a large profit.  In a parallel action, the United States Attorney's Office for the Eastern District of New York announced criminal charges against the defendants.  On March 4, the SEC announced the entry of an order suspending trading in the securities of HD View and for West Coast, pending the outcome of a parallel criminal action by the Federal Bureau of Investigation.  Following the announcement of the suspension, HD View stock prices and volume increased significantly, and the defendants agreed to pay over $1.5 million in disgorgement, prejudgment interest, penalties, and an officer and director bar.  Beaufort agreed to settle the charges without admitting or denying the allegations of the complaint, and to pay a $1 million civil penalty. The SEC's investigation, which is continuing, has been conducted by Patrick McCluskey and Cheryl Crumpton of the SEC Enforcement Division's Market Abuse Unit in the New York Regional Office.Â  The SEC appreciates the assistance of the Financial Industry Regulatory Authority of the United Kingdom, the Canadian Securities Commission, the Alberta Securities Commission and the Ontario Securities Commission.


loading configuration file config.json from cache at /home/ec2-user/.cache/huggingface/hub/models--nsi319--legal-led-base-16384/snapshots/d1c0c7730126e04e5c1efd991ef78f4eb0513def/config.json
Model config LEDConfig {
  "_name_or_path": "nsi319/legal-led-base-16384",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "LEDForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_window": [
    1024,
    1024,
    1024,
    1024,
    1024,
    1024
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"


In [32]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-led-base-16384")  
model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-led-base-16384")

padding = "max_length" 

text = 

input_tokenized = tokenizer.encode(text, return_tensors='pt',padding=padding,pad_to_max_length=True, max_length=6144,truncation=True)
summary_ids = model.generate(input_tokenized,
                                  num_beams=4,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2,
                                  min_length=350,
                                  max_length=500)
summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]


SyntaxError: invalid syntax (880312218.py, line 8)